In [15]:
import pandas as pd
import json 
from bs4 import BeautifulSoup

In [16]:
"Data Directory"
js= 'user_data.json'
xml='user_data.xml'
csv='user_data.csv'

In [17]:
"Parse the Json data to a dataframe"
fob=open(js, 'r')
fob=fob.read()
json_data=pd.read_json(fob)
json_data.head(2)


,firstName,lastName,age,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,address_postcode,debt
0,Janet,Whittaker,79,GB06TIPX06791401324359,213175641545275,596,12/17,08/20,Studio 6 Robin court,Christopherland,N49 2LB,NaN
1,Kieran,Heath,83,GB28VQOG22478273510511,4153200250709345,248,11/13,08/21,13 Hale light,East Mandy,BA1 4GB,NaN


In [18]:
"Parse the xml data to a dataframe"
fob=open(xml, 'r')
fob=fob.read()
Soup=BeautifulSoup(fob,'html.parser')
users=Soup.findAll('user')

lastname=[]
firstname=[]
address_postcode=[]
age=[]
commute_distance=[]
company=[]
dependants=[]
marital_status=[]
pension=[]
retired=[]
salary=[]
sex=[]

for user in users:
  lastname.append(user.get('lastname'))
  firstname.append(user.get('firstname'))
  address_postcode.append(user.get('address_postcode'))
  age.append(user.get('age'))
  commute_distance.append(user.get('commute_distance'))
  company.append(user.get('company'))
  dependants.append(user.get('dependants'))
  marital_status.append(user.get('marital_status'))
  pension.append(user.get('pension'))
  retired.append(user.get('retired'))
  salary.append(user.get('salary'))
  sex.append(user.get('sex'))

xml_data=pd.DataFrame([firstname,lastname, address_postcode, age, commute_distance,company,dependants, marital_status, \
                 pension, retired, salary,sex]).T
                
xml_data.columns=['firstname','lastname', 'address_postcode', 'age', 'commute_distance','company','dependants',\
            'marital_status', 'pension', 'retired', 'salary','sex']

xml_data.head(2)

,firstname,lastname,address_postcode,age,commute_distance,company,dependants,marital_status,pension,retired,salary,sex
0,Hannah,Jones,N06 4LG,21,6.56,Ward and Sons,2,married or civil partner,0,False,20603,Female
1,Tracy,Rowley,M1 6JD,50,11.01,"Fuller, King and Robinson",1,single,0,False,39509,Female


In [19]:
"parse CSV data to a dataframe"
csv_data=pd.read_csv(csv)
csv_data.head()

,First Name,Second Name,Age (Years),Sex,Vehicle Make,Vehicle Model,Vehicle Year,Vehicle Type
0,Oliver,Brady,68,Male,Mitsubishi,WRX,2003,Sedan
1,Denis,Jackson,35,Male,Toyota,Canyon Regular Cab,2011,Convertible
2,Joshua,Williams,26,Male,Honda,1500 Crew Cab,1999,Sedan
3,Malcolm,Johnson,37,Male,Dodge,Legacy,1996,"Sedan, Wagon"
4,Ashley,Bates,57,Male,Chevrolet,A8,2018,Sedan


In [20]:
"Sort the respective data by their Firstname"
json_data=json_data.sort_values('firstName').reset_index().drop('index', axis=1)
csv_data=csv_data.sort_values('First Name').reset_index().drop('index', axis=1)
xml_data=xml_data.sort_values('firstname').reset_index().drop('index', axis=1)

"Create new column Full Name to serve as our Primary ID"
json_data['Full Name']=json_data['firstName']+ ' ' + json_data['lastName']
json_data['Full Name']=json_data['Full Name'].str.upper()
csv_data['Full Name']=csv_data['First Name']+ ' ' + csv_data['Second Name']
csv_data['Full Name']=csv_data['Full Name'].str.upper()
xml_data['Full Name']=xml_data['firstname']+ ' ' + xml_data['lastname']
xml_data['Full Name']=xml_data['Full Name'].str.upper()

In [21]:
"Combine the three dataframes to one using merge function"
xml_csv=xml_data.merge (csv_data, how='outer', on='Full Name')
Data=xml_csv.merge(json_data, how='outer', on='Full Name')

In [22]:
"Drop duplicate columns;FirstName, LastName, address_postcode, age, Sex"
Data=Data.drop(['address_postcode_y', 'age_x', 'age_y', 'sex', 'First Name','Second Name', 'firstName', 'lastName'], axis=1)

In [23]:
Data.head(10)

,firstname,lastname,address_postcode_x,commute_distance,company,dependants,marital_status,pension,retired,salary,Full Name,Age (Years),Sex,Vehicle Make,Vehicle Model,Vehicle Year,Vehicle Type,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,debt
0,Aaron,Marsh,HP3 6LF,0,N/A,2,married or civil partner,21374,True,74887,AARON MARSH,84,Male,Lexus,Windstar Passenger,1994,SUV,GB32NZWM83573155733294,6512070056375422,1666,10/13,07/18,308 Crawford camp,West Tom,NaN
1,Aaron,Kay,SP9E 9JT,19.73,Rowley Inc,2,single,0,False,94214,AARON KAY,41,Male,BMW,Savana 1500 Passenger,2014,Hatchback,GB57QBES65345309601440,4342035909508,5837,10/18,10/25,Studio 2 Vaughan ridge,South Allan,NaN
2,Aaron,Thomas,EH7 9ZQ,11.3,Farmer and Sons,2,married or civil partner,0,False,40932,AARON THOMAS,32,Male,Ford,QX80,2002,Sedan,GB62LUUJ66554564649294,30518213759025,995,12/18,07/22,1 Bryan vista,Darrentown,NaN
3,Abbie,Doyle,HG5 6ZH,6.57,Anderson PLC,1,single,0,False,20611,ABBIE DOYLE,26,Female,Ford,Sportage,2017,Coupe,GB78IDJR05415267151333,180068055911498,530,02/18,09/23,Flat 20 Alice mission,South Juliefurt,NaN
4,Abbie,Hawkins,BS27 7WU,9.38,Young Inc,3,single,0,False,31848,ABBIE HAWKINS,66,Female,Mercury,FJ Cruiser,2019,"Convertible, Coupe",GB09CZVS72234394595057,342556174236362,217,03/20,10/29,Studio 99 Wall crescent,East Trevorshire,NaN
5,Abbie,O'Brien,B48 5PB,15.27,"Carroll, Bennett and Roberts",4,single,0,False,63322,ABBIE O'BRIEN,58,Female,Ford,Aspire,2008,Pickup,GB38GNMY44055200590427,4936645026502,503,02/16,05/30,90 Kirsty harbor,Frederickhaven,NaN
6,Abbie,Ahmed,B9J 7LX,16.12,"Jones, Macdonald and Cunningham",2,single,0,False,68707,ABBIE AHMED,26,Female,Cadillac,Ram Van 3500,1997,Coupe,GB25PPSP61467167054880,213166626571378,995,06/15,10/27,018 Kelly turnpike,Tomborough,7543.96
7,Abigail,Barnes,B11 3DL,0,N/A,,single,26463,True,79602,ABIGAIL BARNES,83,Female,Lincoln,Vandura G3500,2017,Van/Minivan,GB03VGZX21412327627116,30244220976995,555,12/16,02/29,Flat 3 Birch route,South Sylviamouth,NaN
8,Abigail,Henderson,M2C 0DG,6.55,"Kirby, Humphreys and Metcalfe",3,married or civil partner,0,False,20535,ABIGAIL HENDERSON,60,Female,Acura,Prius,1999,Pickup,GB48ZUPY79191875147710,4763549686049268,497,04/15,06/28,8 Brookes land,Robinsonshire,NaN
9,Abigail,Richardson,BB8W 7RX,7.87,"Williamson, Rice and Potter",4,married or civil partner,0,False,25510,ABIGAIL RICHARDSON,60,Female,Toyota,Grand Caravan Passenger,2007,Sedan,GB93HDDN99839847681775,4144189218167278,162,05/16,06/19,2 Victoria well,Butlershire,11566.64


# **# Utilizing Information from the text file**

In [24]:
text='user_data.txt'
fob=open(text, 'r')
text=fob.read()
print(text)

"Debra Wood phoned up at the weekend. They mentioned that they might have gotten their security code incorrect on their billing information, numbers in the wrong order or something. They think that it's 592"
"Congratulations on the promotion Howard! Keep up the good work. Enjoy the extra £2000 salary, just make sure not to spend it all like last time Hilary. ;) Remember we still work for Hussain-Adams, at least until you win that yacht. See you on Monday"
"Happy Birthday Ms Molly Dobson! We can't believe you're turning 82 today. Make sure to check out our latest offers on Home Appliances, offer ends midnight."
"INTERNAL MEMO (CONFIDENTIAL): Harrison, we've just found a pretty serious error in Mr Miller's file. He shouldn't be on £24515 for his annual pension. This needs rectifying immediately or we're in big trouble! I think it's either £27334 or £25901... just put the higher one in and hope nobody notices."


Reading through the text file, the following infromation was extracted and will be corrected on the unified record:


1.   Debra Wood security code is to be corrected to (592)
2.   Howard Hilary who works for Hussain-Adams new salary increases by 2000 
3.   Ms Molly Dobson is now 82
4.   Mr Miller whose pension was 24515 should be corrected to 27334



In [25]:
"view the data value BEFORE making changes"
print(Data.loc[Data['Full Name']=="DEBRA WOOD", 'credit_card_security_code'])
print(Data.loc[Data['Full Name']=="HILARY HOWARD", 'salary'])
print(Data.loc[Data['Full Name']=="MOLLY DOBSON", 'Age (Years)'])
print(Data.loc[Data['Full Name']=="LEWIS MILLER", 'pension'])

269    952
Name: credit_card_security_code, dtype: int64
415    84396
Name: salary, dtype: object
783    81
Name: Age (Years), dtype: int64
673    24515
Name: pension, dtype: object


In [26]:
"Debra Wood"
Data.loc[Data['Full Name']=="DEBRA WOOD", 'credit_card_security_code']= 592
"Howard Hilary"
Data.loc[Data['Full Name']=="HILARY HOWARD", 'salary']= int(Data.loc[Data['Full Name']=="HILARY HOWARD", 'salary'])+ 2000
"Ms Molly Dobson"
Data.loc[Data['Full Name']=="MOLLY DOBSON", 'Age (Years)']=82
"Mr Miller"
Data.loc[(Data['lastname']=="Miller") &  (Data['pension']=="24515"), 'pension']= '27334'

In [27]:
"view the data value AFTER making changes"
print(Data.loc[Data['Full Name']=="DEBRA WOOD", 'credit_card_security_code'])
print(Data.loc[Data['Full Name']=="HILARY HOWARD", 'salary'])
print(Data.loc[Data['Full Name']=="MOLLY DOBSON", 'Age (Years)'])
print(Data.loc[Data['Full Name']=="LEWIS MILLER", 'pension'])

269    592
Name: credit_card_security_code, dtype: int64
415    86396
Name: salary, dtype: object
783    82
Name: Age (Years), dtype: int64
673    27334
Name: pension, dtype: object


Connecting to Writing the unified Record(Data) to DB

In [ ]:
pip install pony

In [ ]:
pip install pymysql

In [ ]:
from pony.orm import *


In [ ]:
db = Database()

In [ ]:
db.bind(provider='mysql', host='europa.ashley.work', user='student_bh95tw', passwd='iE93F2@8EhM@1zhD&u9M@K', db='student_bh95tw')

In [ ]:
class Dataframe1(db.Entity):
  firstname=Required(str)
  lastname=Required(str)
  address_postcode=Required(str)
  commute_distance=Required(str)
  company=Required(str)
  dependants=Required(str)
  marital_status=Required(str)	
  pension	=Required(str)
  retired	=Required(str)
  salary=Required(str)	
  Full_Name= Required(str)	
  Age	= Required(int)
  Sex=Required(str)
  Vehicle_Make=Required(str)
  Vehicle_Model=Required(str)
  Vehicle_Year=Required(int)	
  Vehicle_Type= Required(str)
  iban=Required(str)
  credit_card_number=Required(str)	
  credit_card_security_code=Required(int)
  credit_card_start_date=Required(str)
  credit_card_end_date=Required(str)	
  address_main=Required(str)
  address_city=Required(str)	
  debt=Required(str)

In [ ]:
db.generate_mapping(create_tables=True)

In [28]:
Data['dependants'].replace('', '0', inplace=True)


In [29]:
New_Data=Data.to_records(index=False).tolist()

In [ ]:
for record in New_Data:
  db.insert("dataframe1", firstname=record[0],lastname=record[1],address_postcode=record[2],commute_distance=record[3],company=record[4],dependants=record[5],\
  marital_status=record[6],	pension	=record[7],retired	=record[8],salary=str(record[9]),	Full_Name= record[10],	Age	=record[11],Sex=record[12],\
  Vehicle_Make=record[13],Vehicle_Model=record[14],Vehicle_Year=record[15],	Vehicle_Type= record[16],iban=record[17],credit_card_number=str(record[18]),\
  credit_card_security_code=record[19],credit_card_start_date=record[20],credit_card_end_date=record[21],	address_main=record[22],
  address_city=record[23],debt=str(record[24]))
  commit()  
    
    
